<a href="https://colab.research.google.com/github/raneye99/COMP0087/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#set up
gdrive_location = "/content/drive/MyDrive/NLP"
pat_file = "pat.json"
import subprocess
import json
from google.colab import drive
import os
import sys


gdrive_mount = '/content/drive'
# requires giving access to google drive account
drive.mount(gdrive_mount)

# change to relevant workspace 
# - google drive dir - where 'pat_file' should be found
gdrive_dir = os.path.join(gdrive_mount, gdrive_location)
work_dir = gdrive_dir

#url for repo
url = ":x-oauth-basic@github.com/raneye99/COMP0087"

#repo directory
repo_dir = os.path.join(gdrive_dir, os.path.basename(url))

# change to working directory
# os.chdir(work_dir)
assert os.path.exists(work_dir), f"workspace directory: {work_dir} does not exist"
os.chdir(work_dir)
# this is a bit long - would be good to reduce
# if git directory does not exist - clone 
if not os.path.exists(repo_dir):
    # get pat file - expected to be in work_dir
    # pat expected to be dict: {"pat": "<personal access token>"}
    with open(os.path.join(gdrive_dir, pat_file), "r+") as f:
        pat = json.load(f)

    # git clone
    print(f"cloning directory: {url}")
    git_clone = subprocess.check_output( ["git", "clone", f"https://{pat['pat']}{url}"] , shell=False)
    print(git_clone)

print(f"changing directory to: {repo_dir}")
os.chdir(repo_dir)

# --
# change branch - review this
# --

try:
    branch_name = "main"
    git_checkout = subprocess.check_output(["git", "checkout", "-t", f"origin/{branch_name}"], shell=False)
    print(git_checkout.decode("utf-8") )
except Exception as e:
    git_checkout = subprocess.check_output(["git", "checkout",  f"{branch_name}"], shell=False)
    print(git_checkout.decode("utf-8") )

# ---
# git pull
# ---
# could be redunant if just clone, but whatever

git_pull = subprocess.check_output(["git", "pull"], shell=False)
print(git_pull.decode("utf-8") )


if repo_dir not in sys.path:
    print(f"adding {repo_dir} to sys.path")
    sys.path.extend([repo_dir])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
changing directory to: /content/drive/MyDrive/NLP/COMP0087
Your branch is up to date with 'origin/main'.

Updating 3319a47..1260e78
Fast-forward
 example.ipynb          | 141 +++++++++++++++++++++++++++++++++++++++++++++++++
 mosei_model_load.ipynb |  22 ++++++--
 mtl/data_utils.py      |  64 ++++++++++++++++++++++
 test.ipynb             |  67 +++++++++++++++++++++++
 4 files changed, 291 insertions(+), 3 deletions(-)
 create mode 100644 example.ipynb
 create mode 100644 mtl/data_utils.py
 create mode 100644 test.ipynb



In [9]:
#import libraries
import pickle
from collections import defaultdict
from urllib.request import urlopen
import numpy as np
import pandas as pd
import os
import h5py
import sys
import requests
import zipfile
import inspect

In [18]:
#load data

#file located here: https://drive.google.com/file/d/1tcVYIMcZdlDzGuJvnMtbMchKIK9ulW1P/view

#make data directory if doesn't exist in path folder
os.makedirs('data', exist_ok=True)

#download meld and mosei zip files for data (mosei is from CMU, meld is friends episodes)
!file=1tcVYIMcZdlDzGuJvnMtbMchKIK9ulW1P && wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='${file} -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="${file} -O data/data.zip && rm -rf /tmp/cookies.txt

#unzip data
!unzip data/data.zip -d data/mosei

--2022-04-01 15:35:47--  https://docs.google.com/uc?export=download&confirm=t&id=1tcVYIMcZdlDzGuJvnMtbMchKIK9ulW1P
Resolving docs.google.com (docs.google.com)... 173.194.217.102, 173.194.217.138, 173.194.217.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.217.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4b9mk72ei7v2j6c72r9suqstvrtbihqs/1648827300000/04146720491471605701/*/1tcVYIMcZdlDzGuJvnMtbMchKIK9ulW1P?e=download [following]
--2022-04-01 15:35:48--  https://doc-08-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4b9mk72ei7v2j6c72r9suqstvrtbihqs/1648827300000/04146720491471605701/*/1tcVYIMcZdlDzGuJvnMtbMchKIK9ulW1P?e=download
Resolving doc-08-a8-docs.googleusercontent.com (doc-08-a8-docs.googleusercontent.com)... 173.194.216.132, 2607:f8b0:400c:c12::84
Connecting to doc-08-a8-docs.googleusercontent.com (doc-

In [19]:
#preprocess

#training data
# print(os.getcwd())
text_tr = pickle.load(open('data/mosei/MOSEI/train_sentences.p',"rb"))
emotion_tr = pickle.load(open('data/mosei/MOSEI/train_emotion.p', "rb"))
sentiment_tr = pickle.load(open('data/mosei/MOSEI/train_sentiment.p', "rb"))
#change sentimement to be binary
sentiment_tr = {k: np.sign(v) for k,v in sentiment_tr.items()}


#valid data
text_val = pickle.load(open('data/mosei/MOSEI/valid_sentences.p',"rb"))
emotion_val = pickle.load(open('data/mosei/MOSEI/valid_emotion.p', "rb"))
sentiment_val = pickle.load(open('data/mosei/MOSEI/valid_sentiment.p', "rb"))
#change sentiement to be binary
sentiment_val = {k: np.sign(v) for k,v in sentiment_val.items()}

/content/drive/MyDrive/NLP/COMP0087
